# A basic FEM example using the new GmshSession class

The new GmshSession class adds another example of how to interact with the Gmsh api in python.

In [ ]:
from ada import Beam, Part, Assembly, Plate, PrimBox, Pipe, PrimSphere
from ada.fem.mesh.gmshapiv2 import GmshSession, GmshOptions
from ada.concepts.structural import make_ig_cutplanes

In [ ]:
bm1 = Beam("MyBeam", (0, 0, 1), (1, 0, 1), "IPE300")
bm2 = Beam("MySecondBeam", (1.01, 0, 1), (2, 0, 1), "IPE300")
bm3 = Beam("MyThirdBeam", (2.01, 0, 1), (3, 0, 1), "IPE300")

pl_atts = dict(origin=(1, 1, 1), xdir=(1, 0, 0), normal=(0, 0, 1))
pl_points = [(0,0), (1,0), (1,1), (0,1)]
pl1 = Plate('MyPlate', pl_points, 10e-3, **pl_atts)


pipe = Pipe("MyPipe", [(0, 0.5, 0), (1, 0.5, 0), (1.2, 0.7, 0.2), (1.5, 0.7, 0.2)], "OD120x6")

p1, p2 = (1, -2, 0), (2, -1, 1)
shp1 = PrimBox("MyBox", p1, p2)
shp1.add_penetration(PrimSphere("MyCutout", p1, 0.5))

with GmshSession(silent=True, options=GmshOptions(Mesh_ElementOrder=2)) as gs:
    gs.add_obj(bm1, "shell")
    solid_bm = gs.add_obj(bm2, "solid")
    gs.add_obj(bm3, "line")
    gs.add_obj(pl1, "shell")
    gs.add_obj(shp1, "solid")
    gs.add_obj(pipe, "shell")
    #gs.open_gui()
    gs.mesh(0.05)
    fem = gs.get_fem()

a = Assembly() / (Part("MyBeam", fem=fem) / [bm1, bm2, bm3, pl1, shp1, pipe])
a